In [1]:
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import childespy
import numpy as np
import os
import sys
import imp
import pandas as pd
import transformers
import torch
import re
import unicodedata
import scipy.stats
import copy
from string import punctuation
from os.path import join, exists

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

In [2]:
sys.path.append('../../')
from src.utils import configuration, load_models, transformers_bert_completions, data_cleaning, phonology
config = configuration.Config()

# Identify "Communicative Successes" and "Communicative Failures" 

Defintions:  
`xxx`: no lexical label, phonetically un-transcribable  
`yyy`: no lexical label, phonetically transcribable  
`Communicative success`: token from utterances with no xxx's or yyy's   
`Communicative failure`: a yyy token from an utterance with no other xxx's or yyy's

Later, we'll further restrict both to tokens that are monosyllabic

In [3]:
pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

R[write to console]: Using supported database version: '2020.1'.



In [4]:
phono_glosses = childespy.get_sql_query('select gloss, target_child_name, target_child_age, \
    speaker_code, actual_phonology, model_phonology, transcript_id, utterance_id, \
    token_order, corpus_name, collection_name, language from token where \
    actual_phonology != "" and model_phonology != "" and speaker_code in ("MOT", "FAT","CHI") and collection_name = "Eng-NA" \
    and corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")

R[write to console]: Using supported database version: '2020.1'.



In [5]:
phono_glosses.corpus_name.value_counts()

Providence    396621
Name: corpus_name, dtype: int64

In [6]:
phono_glosses.loc[phono_glosses.gloss == 'xxx'].actual_phonology.value_counts()

*          26736
ə             10
(.)            7
aɪ             4
pitched        2
           ...  
maɪ            1
tʃid           1
faɪ            1
noises         1
dʒeɪ           1
Name: actual_phonology, Length: 76, dtype: int64

Actual phonology is almost always * for xxx items

In [7]:
phono_glosses.loc[phono_glosses.gloss == 'yyy'].actual_phonology.value_counts()

ɛ           3206
ʌ           2132
ɪ           1881
ə            512
o            507
            ... 
wɑjɪ           1
juhæhɪɡə       1
bəhɑʊ          1
waɪtæ          1
iːɪːoː         1
Name: actual_phonology, Length: 30293, dtype: int64

Actual phonology *is* populated for tokens with `yyy` lexical label

In [8]:
chi_phono = phono_glosses.loc[(phono_glosses.speaker_code == 'CHI') & 
    (phono_glosses.target_child_age < (365*5))]

In [9]:
def count_transmission_errors(utt_vector, error_codes):
    return(np.sum([x in error_codes for x in  utt_vector]))

In [10]:
xxxs_per_utt = chi_phono.groupby('utterance_id').gloss.agg(
    lambda x: count_transmission_errors(x, ['xxx'])).reset_index()
xxxs_per_utt.columns = ['utterance_id', 'num_xxx']
yyys_per_utt = chi_phono.groupby('utterance_id').gloss.agg(
    lambda x: count_transmission_errors(x, ['yyy'])).reset_index()
yyys_per_utt.columns = ['utterance_id', 'num_yyy']
failures_per_utt = xxxs_per_utt.merge(yyys_per_utt)

In [11]:
yyy_utts = failures_per_utt.loc[(failures_per_utt.num_xxx == 0) &  (failures_per_utt.num_yyy == 1)]

In [12]:
yyy_utts.shape

(31457, 3)

In [13]:
success_utts = failures_per_utt.loc[(failures_per_utt.num_xxx == 0) &  
    (failures_per_utt.num_yyy == 0)]

In [14]:
success_utts.shape

(83880, 3)

In [15]:
tokens_from_errorless_utts = chi_phono.loc[chi_phono.utterance_id.isin(success_utts.utterance_id)]

In [16]:
#exclude un-transcribed tokens and syllabically transcribed tokens
excludes = ['*','(.)','(..)', '(...)','(....)','(.....)']
tokens_from_errorless_utts = tokens_from_errorless_utts.loc[~(tokens_from_errorless_utts.actual_phonology.isin(excludes) |
    tokens_from_errorless_utts.model_phonology.isin(excludes))]

In [17]:
tokens_from_errorless_utts.shape

(214239, 12)

In [18]:
# example phonology
tokens_from_errorless_utts.actual_phonology

1          ɑmɪ
3          wiː
4          wiː
5           uː
52           ɛ
          ... 
396606       o
396607     waɪ
396608     liʔ
396609       ɪ
396610    hɪpo
Name: actual_phonology, Length: 214239, dtype: object

In [19]:
# 31,457 transmission errors (from 31,457 utterances)
# 214,239 transmission successes (from 83,880 utterances)
# this will be further decreased later by the need to test monosyllabic forms

# Load BERT Models + CMU Dict

### Get the CMU Pronunciation Dictionary

In [20]:
import imp
imp.reload(configuration)
config = configuration.Config()

In [21]:
cmu_in_childes = pd.read_pickle(config.cmu_path)
cmu_2syl_inchildes = cmu_in_childes.loc[cmu_in_childes.num_vowels <=2]

In [22]:
cmu_in_childes.shape

(8943, 8)

# Prep Utterances / Tokens for BERT

In [23]:
# Load utterances from the Providence corpus from childs-db

this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')
if config.regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, target_child_name, speaker_code, type from utterance where speaker_code in ("MOT", "FAT","CHI") and corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



In [24]:
utt_glosses = utt_glosses.rename(columns = {'id' : 'utterance_id'})

In [26]:
# Prep the utterances for tokenization with BERT. Importantly, add back the appropriate punctutation for the sentence type
utt_glosses = utt_glosses[~utt_glosses.gloss.isna()]
utt_glosses = data_cleaning.clean_glosses_and_add_speaker_label(utt_glosses, True)

In [27]:
utt_glosses.head(5)

,gloss,transcript_id,utterance_id,utterance_order,target_child_name,speaker_code,type,punct,speaker_code_simple,gloss_with_punct
1,anywhere you'll feel comfortable um anywhere,42204,16759261,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...
2,please don't do that,42204,16759270,3,Alex,MOT,declarative,.,[CGV],[CGV] please don't do that.
3,this is,42204,16759279,4,Alex,MOT,self interruption,.,[CGV],[CGV] this is.
4,Mommy,42204,16759315,6,Alex,CHI,declarative,.,[CHI],[CHI] mommy.
5,okay that's fine,42204,16759322,7,Alex,MOT,declarative,.,[CGV],[CGV] okay that's fine.


### Build the Vocabulary

In [28]:
import imp
imp.reload(load_models)
imp.reload(transformers_bert_completions)

<module 'src.utils.transformers_bert_completions' from '/home/stephan/notebooks/child-directed-listening/src/tier_1/../../src/utils/transformers_bert_completions.py'>

In [29]:
initial_tokenizer = load_models.get_primary_tokenizer()
initial_vocab, cmu_in_initial_vocab, cmu_indices_for_initial_vocab = load_models.get_initial_vocab_info(initial_tokenizer)

In [32]:
print('Vocab mask contains '+str(len(initial_vocab))+' entries (written words)') 
print('CMU pronunciation dictionary has '+str(cmu_in_initial_vocab.shape[0])+ ' entries')
print('CMU pronunciation dictionary may have multiple entries for each word')

Vocab mask contains 7997 entries (written words)
CMU pronunciation dictionary has 8943 entries
CMU pronunciation dictionary may have multiple entries for each word


In [33]:
# confirm yyy treated as a separate character
if 'yyy' not in initial_tokenizer.tokenize('this is a yyy.'):
    raise ValueError('yyy must be treated as a separate character; check the tokenizer')

In [34]:
# build a dataframe of tokens 
# this is slow, because tokenization is slow and single-core
def inflate(row):
    tokens = initial_tokenizer.tokenize(row['gloss_with_punct'])
    return(pd.DataFrame({'token':tokens, 'utterance_id':row['utterance_id']}) )

inflate_path = join(config.prov_csv_dir, 'pvd_utt_glosses_inflated.csv')
if config.regenerate:
    all_tokens = pd.concat([inflate(x) for x in utt_glosses.to_dict('records')])
    all_tokens = all_tokens.merge(utt_glosses)
    all_tokens.to_csv(inflate_path)

else:
    all_tokens = pd.read_csv(inflate_path, na_filter=False)

In [48]:
all_tokens = all_tokens.sort_values(by= ['transcript_id','utterance_order'])

In [46]:
# Assign a token_id (integer in the BERT vocabulary). 
# Because these are from the tokenized utterances, there is no correpsondence with childes-db token ids.
all_tokens['token_id'] = initial_tokenizer.convert_tokens_to_ids(all_tokens['token'])


In [50]:
all_tokens.iloc[0:20]

,token,utterance_id,gloss,transcript_id,utterance_order,target_child_name,speaker_code,type,punct,speaker_code_simple,gloss_with_punct,token_id
0,[cgv],16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,30523
1,anywhere,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,5973
2,you,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,2017
3,',16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,1005
4,ll,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,2222
5,feel,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,2514
6,comfortable,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,6625
7,um,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,8529
8,anywhere,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,5973
9,.,16759261,anywhere you'll feel comfortable um anywhere,42204,2,Alex,MOT,declarative,.,[CGV],[CGV] anywhere you'll feel comfortable um anyw...,1012


### Add back IPA, syllable structure, and child ages for child productions

In [52]:
 %%capture
# get the token-level data, esp phonology

save_phono_inflated_path = join(config.prov_csv_dir, 'pvd_utt_glosses_phono_inflated.pkl')

if config.regenerate:

    # get token-level information for Providence
    pvd_chi_tokens = childespy.get_sql_query('select gloss, target_child_name, target_child_age, \
    speaker_code, actual_phonology, model_phonology, transcript_id, utterance_id, \
    token_order from token where speaker_code = "CHI" and corpus_id = '+str(pvd_idx),
        db_version = "2020.1")
    pvd_chi_tokens['gloss'] = [data_cleaning.fix_gloss(x) for x in pvd_chi_tokens.gloss]
    
    # prep the tokens generated from segmenting the utterances
    all_tokens_test = copy.deepcopy(all_tokens) 

    # initialize the fields that need to be populated
    all_tokens_test['actual_phonology'] = ''
    all_tokens_test['model_phonology'] = ''
    all_tokens_test['target_child_age'] = np.nan
    
    # get a set of unique utterances
    _, idx = np.unique(all_tokens_test.utterance_id, return_index=True)
    all_utt_indices = all_tokens_test.utterance_id[np.sort(idx)]
    
    # For fast retrieval of IPA, split pvd_chi_tokens into a dictionary
    pvd_chi_tokens_list = pvd_chi_tokens.groupby(['utterance_id'])
    pvd_chi_tokens_dict = dict(zip(
        [x[0] for x in pvd_chi_tokens_list], 
        [x[1] for x in pvd_chi_tokens_list], 
    ))
    
    # For fast retrival of BERT tokenization
    all_tokens_test_list = all_tokens_test.groupby(['utterance_id'])
    all_tokens_test_dict = dict(zip(
        [x[0] for x in all_tokens_test_list], 
        [x[1] for x in all_tokens_test_list], 
    ))
        
    # Augment the tokens from all_tokens with the IPA from pvd_chi_tokens 
    rvs = [] 
    utts_to_retrieve = yyy_utts.utterance_id.to_list() + success_utts.utterance_id.to_list()
    i=-1
    for utt_index in all_utt_indices: #utts_to_retrieve: #[16760331]:       
        i+=1
        if i % int(len(all_utt_indices) / 100) == 0:
            print(str(np.round((i / (len(all_utt_indices)) * 100),2))+'% complete...')    
            # should learn to use tqdm instead
        if utt_index in utts_to_retrieve:        
            utt_df = copy.deepcopy(all_tokens_test_dict[utt_index])
            utt_df['model_phonology'] = transformers_bert_completions.augment_with_ipa(
              utt_df, pvd_chi_tokens_dict[utt_index],initial_tokenizer, 'model_phonology')
            utt_df['actual_phonology'] = transformers_bert_completions.augment_with_ipa(
              utt_df, pvd_chi_tokens_dict[utt_index],initial_tokenizer, 'actual_phonology')
            utt_df['target_child_age'] = pvd_chi_tokens_dict[utt_index].iloc[0].target_child_age    
            rvs.append(utt_df)  
        else:
            rvs.append(all_tokens_test_dict[utt_index])  
            
    # get the resulting augmented forms back into a dataframe
    all_tokens_phono = pd.concat(rvs)
    
    # add a unique identifier to the BERT tokens
    all_tokens_phono['bert_token_id'] = range(all_tokens_phono.shape[0])
    
    #save the results
    all_tokens_phono.to_pickle(save_phono_inflated_path)
else:
    all_tokens_phono = pd.read_pickle(save_phono_inflated_path)

In [53]:
# Inspect the IPA
all_tokens_phono.loc[all_tokens_phono.actual_phonology != ''][['token','actual_phonology','model_phonology']]

,token,actual_phonology,model_phonology
23,mommy,ɑmɪ,mɑmiː
55,yyy,ʌ,*
125,wee,wiː,wiː
128,yyy,aʊ,*
151,wee,wiː,wiː
...,...,...,...
2991828,nobody,nobɑɾi,noʊbɑdiː
2991829,hates,heɪs,heɪts
2991834,oh,o,oʊ
2991835,why,waɪ,waɪ


In [55]:
cleaned_inflated_save = join(config.prov_csv_dir, 'pvd_utt_glosses_phono_cleaned_inflated.pkl')

if config.regenerate:    

    # Do the same excludes as were used to identify appropriate utterances
    excludes = ['*','(.)','(..)', '(...)','(....)','(.....)']
    all_tokens_phono.loc[all_tokens_phono.actual_phonology.isin(excludes),'actual_phonology'] =''
    all_tokens_phono.loc[all_tokens_phono.actual_phonology.str.contains('V'),'actual_phonology'] =''
    
    # remap phonology from narrow phonetic transcription to broad phonological transcription
    all_tokens_phono['model_phonology_clean'] = [phonology.phone_remap(x) for x in all_tokens_phono['model_phonology']]
    all_tokens_phono['actual_phonology_clean'] = [phonology.phone_remap(x) for x in all_tokens_phono['actual_phonology']]

    # remove any non-combining diacritical marks
    all_tokens_phono['model_phonology_no_dia'] = [phonology.strip_accents(x) for x in \
    all_tokens_phono['model_phonology_clean']]
    all_tokens_phono['actual_phonology_no_dia'] = [phonology.strip_accents(x) for x in \
    all_tokens_phono['actual_phonology_clean']]
    
    
    # 8/13/21: Changes to limit num vowels on both actual and model phonology.
    # Note will not regen this until later, even if this shows up in near future commits

    all_tokens_phono = data_cleaning.assign_num_vowels_per_phonology(all_tokens_phono, 'actual')
    all_tokens_phono = data_cleaning.assign_num_vowels_per_phonology(all_tokens_phono, 'model')
    
    all_tokens_phono = data_cleaning.combine_num_vowels_phonology(all_tokens_phono)
    
    all_tokens_phono.to_pickle(cleaned_inflated_save)
else:
    all_tokens_phono = pd.read_pickle(cleaned_inflated_save)


Currently handling * in IPA by dropping from consideration in num_vowels.
Currently handling * in IPA by dropping from consideration in num_vowels.


In [56]:
all_tokens_phono.loc[all_tokens_phono.actual_phonology_no_dia != '']['actual_phonology_no_dia']

23            ɑmə
55              ə
125            wi
128            aʊ
151            wi
            ...  
2991828    nobɑɾi
2991829      heəs
2991834         o
2991835       waə
2991836       liʔ
Name: actual_phonology_no_dia, Length: 251243, dtype: object

In [57]:
all_tokens_phono.shape

(4189555, 27)

### Identify the tokens that can be evaluated 

In [39]:
# find the tokens in the resulting dataframe that belong to the utterances identified above

In [58]:
initial_vocab

array(['a', 'b', 'c', ..., 'hideout', 'pudding', 'stalks'], dtype='<U18')

In [59]:

successful_utt_ids = set(success_utts['utterance_id'])

initial_vocab_set = set(initial_vocab)

yyy_utt_ids = set(yyy_utts['utterance_id'])

all_tokens_phono['in_vocab'] = all_tokens_phono['token'].isin(initial_vocab_set)

# 8/1/21: Changed this line to include the vocab constraint.
all_tokens_phono['success_token'] = [(x in successful_utt_ids) and (y) for x, y in 
    zip(all_tokens_phono['utterance_id'], all_tokens_phono['in_vocab'])]
# end changes

all_tokens_phono['yyy_token'] = [x in yyy_utt_ids for x in 
    all_tokens_phono['utterance_id']]


In [60]:
all_tokens_phono.shape

(4189555, 30)

In [61]:
assert '' not in set(all_tokens_phono[all_tokens_phono['num_vowels'] <= 2].actual_phonology)

### Identify the subset of success and failure utterances that have transcriptions

In [62]:
all_tokens_phono['partition'] = 'none'

In [63]:
success_tokens = all_tokens_phono.loc[(all_tokens_phono['success_token']) & 
    (all_tokens_phono['num_vowels'] <= 2) ]
all_tokens_phono.loc[(all_tokens_phono['success_token']) & 
    (all_tokens_phono['num_vowels'] <= 2), 'partition'] = 'success'     
success_tokens.shape

(187068, 31)

In [64]:
# Successes conditions

success_tokens_check = all_tokens_phono[all_tokens_phono.partition == 'success']
assert all(success_tokens_check['in_vocab'])
assert all(success_tokens_check.utterance_id.isin(successful_utt_ids))
assert all(success_tokens_check['num_vowels'] <= 2)


In [65]:
all_tokens_phono.loc[(all_tokens_phono['success_token']) & 
    (all_tokens_phono['num_vowels'] <= 2)]

,token,utterance_id,gloss,transcript_id,utterance_order,target_child_name,speaker_code,type,punct,speaker_code_simple,...,cv_collapsed_actual,num_vowels_actual,cv_raw_model,cv_collapsed_model,num_vowels_model,num_vowels,in_vocab,success_token,yyy_token,partition
23,mommy,16759315,Mommy,42204,6,Alex,CHI,declarative,.,[CHI],...,vcv,2.0,cvcv,cvcv,2.0,2.0,True,True,False,success
125,wee,16759467,wee,42204,24,Alex,CHI,declarative,.,[CHI],...,cv,1.0,cv,cv,1.0,1.0,True,True,False,success
151,wee,16759501,wee,42204,28,Alex,CHI,declarative,.,[CHI],...,cv,1.0,cv,cv,1.0,1.0,True,True,False,success
1255,blue,16759917,blue,42209,28,Alex,CHI,declarative,.,[CHI],...,vcv,2.0,ccv,cv,1.0,2.0,True,True,False,success
1461,uh,16759987,uh,42209,34,Alex,CHI,question,?,[CHI],...,cv,1.0,v,v,1.0,1.0,True,True,False,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991812,help,17280876,help,42569,751,William,CHI,declarative,.,[CHI],...,vc,1.0,cvcc,cvc,1.0,1.0,True,True,False,success
2991829,hates,17280946,nobody hates Simba,42569,755,William,CHI,declarative,.,[CHI],...,cvc,1.0,cvvcc,cvc,1.0,1.0,True,True,False,success
2991834,oh,17280964,oh why lick hippo,42569,756,William,CHI,self interruption,.,[CHI],...,v,1.0,vv,v,1.0,1.0,True,True,False,success
2991835,why,17280964,oh why lick hippo,42569,756,William,CHI,self interruption,.,[CHI],...,cv,1.0,cvv,cv,1.0,1.0,True,True,False,success


In [66]:
yyy_tokens = all_tokens_phono.loc[(all_tokens_phono['yyy_token']) & 
(all_tokens_phono['token'] == 'yyy') & (all_tokens_phono.num_vowels <= 2) ]
all_tokens_phono.loc[(all_tokens_phono['yyy_token']) & 
(all_tokens_phono['token'] == 'yyy') & (all_tokens_phono.num_vowels <= 2),'partition'] = 'yyy'
yyy_tokens.shape

(18233, 31)

In [67]:
all_tokens_phono.partition.value_counts()

none       3984254
success     187068
yyy          18233
Name: partition, dtype: int64

In [68]:
initial_tokenizer.unk_token_id

100

In [69]:
all_tokens_phono.loc[all_tokens_phono.token == 'xxx','token_id'] = initial_tokenizer.unk_token_id
all_tokens_phono.loc[all_tokens_phono.token == 'yyy','token_id'] = initial_tokenizer.unk_token_id

In [70]:
# this adds the partition information
final_save_path = join(config.prov_csv_dir, 'pvd_utt_glosses_phono_cleaned_inflated_to_next_notebook.pkl')
all_tokens_phono.to_pickle(final_save_path)

# Prevalence of Successes and Failures Across Time

In [72]:
# get number of tokens per age
success_utts['set'] = 'success'
yyy_utts['set'] = 'failure'

<ipython-input-72-e26fc16c2e1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  success_utts['set'] = 'success'
<ipython-input-72-e26fc16c2e1c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yyy_utts['set'] = 'failure'


In [73]:
# get child age in days associated with each utterance id and plot it

In [75]:
utt_age = chi_phono.groupby('utterance_id').target_child_age.agg(lambda x: np.unique(x)[0]).reset_index()

In [76]:
utts_with_ages = pd.concat([success_utts, yyy_utts]).merge(utt_age)

In [77]:
utts_with_ages['year'] = .5*np.floor(utts_with_ages['target_child_age'] / (365. /2) ) 
print(utts_with_ages.loc[utts_with_ages.set == 'failure'].year.value_counts())
print(utts_with_ages.loc[utts_with_ages.set == 'success'].year.value_counts())

1.5    9919
2.0    7261
1.0    6693
2.5    4895
3.0    2097
3.5     414
0.5     167
4.0      11
Name: year, dtype: int64
2.0    22432
2.5    21194
1.5    16798
3.0    12564
1.0     6697
3.5     3683
4.0      379
0.5      133
Name: year, dtype: int64


In [78]:
final_utts_save_path = join(config.prov_csv_dir, 'utts_with_ages.csv')
utts_with_ages.to_csv(final_utts_save_path)

In [79]:
assert 'yyy' not in set(success_tokens_check['token'])
# This was the problem that was observed in my iteration of the code.

In [80]:
from datetime import datetime
print(datetime.today())

2023-01-14 09:35:53.843803
